In [ ]:
import os
from  importlib import reload
os.chdir("/home/jovyan/work/data/nasa_planet_gan/")
import loadfunctions

from PIL import Image
import os, sys, glob
import fnmatch
import numpy as np
import tensorflow as tf
import random
from IPython.display import clear_output

from loadfunctions import *
import k_means_plus


In [2]:
source_images_dir = "./source_images/"
image_dir= "./source_resized"
src_resized_np = resize_and_load_image_list(get_image_list(source_images_dir),256,192)

creating resized np image array


In [3]:

print(src_resized_np.shape)

(98, 192, 256, 3)


In [4]:
reload(k_means_plus)
reload(loadfunctions)
test = src_resized_np[0:50]
print(test.shape)
C, S =k_means_plus.compute(test, 5)

(50, 192, 256, 3)


In [21]:
a = get_best_k_for_images(src_resized_np, 3,40,verbose=2)

In [24]:

print(a[0])
for k,score,groups in a[1]:
    print(k, score)
    for group in groups:
        print(len(group))

    if k == a[0]:
        groups_to_use=groups

groups_to_use

8
3 0.2562615955473099
89
1
8
4 0.2568027210884354
85
1
3
9
5 0.16510941726088024
84
1
3
3
7
6 0.7559704732957013
48
1
5
22
3
19
7 0.5178571428571429
57
1
3
4
19
7
7
8 0.808843537414966
31
1
3
13
4
6
17
23
9 0.29474097331240184
24
1
2
3
3
3
3
53
6
10 0.2717687074829932
61
1
3
1
2
2
16
7
4
1
11 0.35946196660482377
22
1
2
3
45
2
4
4
1
2
12
12 0.32307160152196474
60
1
2
2
3
1
4
9
3
2
1
10
13 0.29471788715486197
35
1
4
2
1
2
2
6
2
4
35
3
1
14 0.4464285714285714
57
1
1
3
2
1
3
5
4
3
1
13
1
3
15 0.3406052076002815
30
1
1
2
3
2
1
1
18
1
1
26
5
4
2
16 0.34925788497217064
45
1
1
1
2
2
1
9
2
6
1
2
1
4
9
11
17 0.3904170363797693
1
1
1
3
1
2
2
9
24
4
2
5
5
1
1
16
20
18 0.40734693877551015
26
1
1
1
2
2
1
2
2
12
5
1
1
1
19
3
8
10
19 0.45442176870748296
4
1
3
1
1
2
1
4
16
2
1
1
12
1
2
16
4
12
14
20 0.47959183673469385
13
1
1
1
1
13
15
1
1
1
2
2
1
10
10
1
2
14
6
2
21 0.5
3
1
1
3
1
1
1
4
10
2
1
1
12
1
2
15
4
12
14
1
8
22 0.36451247165532885
37
1
1
1
2
1
1
2
1
2
1
1
4
8
7
6
7
1
1
8
4
1
23 0.271730914588

[[62,
  1,
  2,
  4,
  9,
  11,
  15,
  20,
  23,
  27,
  35,
  40,
  42,
  47,
  48,
  49,
  51,
  52,
  57,
  58,
  69,
  70,
  74,
  79,
  80,
  81,
  82,
  83,
  85,
  86,
  90],
 [94],
 [26, 53, 56],
 [91, 3, 6, 8, 25, 37, 55, 63, 68, 87, 88, 89, 93],
 [61, 17, 38, 64],
 [44, 5, 36, 45, 54, 78],
 [0, 7, 14, 19, 24, 28, 29, 33, 41, 46, 66, 73, 76, 92, 95, 96, 97],
 [18,
  10,
  12,
  13,
  16,
  21,
  22,
  30,
  31,
  32,
  34,
  39,
  43,
  50,
  59,
  60,
  65,
  67,
  71,
  72,
  75,
  77,
  84]]

In [25]:
(import multiprocessing
scores= []

def get_score_for_num_centroids(X,num_centroids):
        C, S =k_means_plus.compute(X, num_centroids)
        groups = get_centroid_groups(C,S)
        lens_of_groups = [len(v) for v in groups]
        score = (num_centroids/(max(lens_of_groups)-min(lens_of_groups)))/num_centroids

        median_length_of_group =np.median(lens_of_groups)
        perfect_mean = len(X)/len(lens_of_groups)
        score = score+(median_length_of_group/perfect_mean)
        return (score,groups)





SyntaxError: invalid syntax (2700782817.py, line 1)

In [ ]:

#use multiprocessing to get the scores for all the centroids
pool = multiprocessing.Pool(processes=8)

#scores = pool.map(get_score_for_num_centroids_test, range(20,50))
scores = pool.starmap(get_score_for_num_centroids, zip(src_resized_np,range(3,10)))



In [19]:

print(scores)
#print("best num of centroids:", np.argmax(scores)+3)

#print("best score:", best_score)
#print("best num_centroids:", best_idx)

[]


In [17]:
# wait for all processes to finish before moving on
pool.close()

ERROR! Session/line number was not unique in database. History logging moved to new session 144


In [18]:
pool.join()

In [11]:
min = 3
for i,score,groups in enumerate(scores):
    print(i+min, score)
    for group in groups:
        print(len(group))

print("best: ", np.argmax(scores)+min)
best = np.argmax(scores)+min

ValueError: attempt to get argmax of an empty sequence

In [ ]:
C, S = k_means_plus.compute(src_resized_np, best)
groups = get_centroid_groups(C,S)

In [23]:

for group in groups:
    print(len(group))

65
1
2
2
1
4
2
9
8
3
1


In [33]:
#create image_dir if it doesn't exist
if not os.path.exists(image_dir):
    os.makedirs(image_dir)
# save each image in each group
for i, group in enumerate(groups_to_use):
    for j, image in enumerate(group):
        print(i,j)
        data = src_resized_np[image].astype(np.uint8)
        print(data.shape)
        im = Image.fromarray(data)
        #im.show()
        im.save(image_dir+"/group_"+str(i)+"_"+str(j)+".png")

0 0
(192, 256, 3)
0 1
(192, 256, 3)
0 2
(192, 256, 3)
0 3
(192, 256, 3)
0 4
(192, 256, 3)
0 5
(192, 256, 3)
0 6
(192, 256, 3)
0 7
(192, 256, 3)
0 8
(192, 256, 3)
0 9
(192, 256, 3)
0 10
(192, 256, 3)
0 11
(192, 256, 3)
0 12
(192, 256, 3)
0 13
(192, 256, 3)
0 14
(192, 256, 3)
0 15
(192, 256, 3)
0 16
(192, 256, 3)
0 17
(192, 256, 3)
0 18
(192, 256, 3)
0 19
(192, 256, 3)
0 20
(192, 256, 3)
0 21
(192, 256, 3)
0 22
(192, 256, 3)
0 23
(192, 256, 3)
0 24
(192, 256, 3)
0 25
(192, 256, 3)
0 26
(192, 256, 3)
0 27
(192, 256, 3)
0 28
(192, 256, 3)
0 29
(192, 256, 3)
0 30
(192, 256, 3)
1 0
(192, 256, 3)
2 0
(192, 256, 3)
2 1
(192, 256, 3)
2 2
(192, 256, 3)
3 0
(192, 256, 3)
3 1
(192, 256, 3)
3 2
(192, 256, 3)
3 3
(192, 256, 3)
3 4
(192, 256, 3)
3 5
(192, 256, 3)
3 6
(192, 256, 3)
3 7
(192, 256, 3)
3 8
(192, 256, 3)
3 9
(192, 256, 3)
3 10
(192, 256, 3)
3 11
(192, 256, 3)
3 12
(192, 256, 3)
4 0
(192, 256, 3)
4 1
(192, 256, 3)
4 2
(192, 256, 3)
4 3
(192, 256, 3)
5 0
(192, 256, 3)
5 1
(192, 256, 3)
5 2


In [ ]:

# _run kmeans on augmented_images.npy
from sklearn.cluster import KMeans
h,w,c = augs[0].shape
augs[0].max()
noise_images = np.random.random_integers(0,255,(1000,h,w,c)).astype(float)

In [ ]:

from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import numpy as np

pca = PCA(2)
noise_PCA = pca.fit_transform(noise_images.reshape(noise_images.shape[0],-1))

pca1 = PCA(2)
augs_PXA = pca1.fit_transform(augs.reshape(augs.shape[0],-1))

pca2 = PCA(2)
noise_and_augs = pca2.fit_transform(np.vstack((augs, noise_images)).reshape(-1,h*w*c))



In [ ]:
import matplotlib.pyplot as plt
def plot_pca(l_pca,n_clusers=4):
        label = KMeans(n_clusters=n_clusers, random_state=0).fit_predict(l_pca)
        plot_l_p(l_pca,label)
        return label,l_pca
def plot_l_p(l_pca,l_label):
        u_labels = np.unique(l_label)
        a = plt.figure(figsize=(20,20))
        for i in u_labels:
            plt.scatter(l_pca[l_label == i , 0] , l_pca[l_label == i , 1] , label = i,alpha=0.5)

        plt.legend()
        plt.show()

In [ ]:
_ = plot_pca(noise_PCA)

In [ ]:
_=plot_pca(augs_PXA,n_clusers=6)

In [ ]:
l,p = plot_pca(noise_and_augs,n_clusers=5)

In [ ]:
l,p = plot_pca(noise_and_augs,n_clusers=2)

In [ ]:

labels = kmeans.labels_
print(labels)
print(kmeans.score(augs[1].reshape(1,-1)))
#plot the kmeans

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
plt.figure(figsize=(10,10))
plt.scatter(augs[:,0], augs[:,1], c=labels, s=50, cmap='viridis')
plt.show()


In [ ]:
rows = []
rows_needed = int(np.ceil(len(augs)/5.0))+1
h,w,c = augs[0].shape
image_size = h*w*c
# print image_size rows_needed h w c
print("images in augs:", len(augs))
print("rows_needed:", rows_needed)
print("image_size:", image_size)
print("h:", h);print("w:", w);print("c:", c)
print (len(augs)/5)
out_np=np.zeros((h*rows_needed,w*5,c))
print("out_np shape:", out_np.shape)

for row in range(rows_needed):
        r_start = row*h
        r_end = (row*h)+h
        print("r_start:", r_start)
        print("r_end:", r_end)
        print(augs[r_start:r_end].shape)
        try:
                out_np[r_start:r_end,:,:] = np.hstack(augs[row*5:(row*5)+5])
        except ValueError as e:
                pass




# vstack the rows
#to_plt = np.vstack(rows)

tf.keras.preprocessing.image.array_to_img(out_np).show()
